In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

from matplotlib import rc
rc("font", family = "Malgun Gothic")

get_ipython().run_line_magic("matplotlib", "inline")

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

import time
from tqdm.notebook import tqdm

from glob import glob
import warnings
warnings.filterwarnings('ignore')

# 1. 파일 불러오기

In [23]:
od_files_2019 = glob("./OD_2019_03/이용객*.xlsx")
od_files_2020 = glob("./OD_2020_03/이용객*.xlsx")
od_files_2021 = glob("./OD_2021_03/이용객*.xlsx")

In [24]:
tmp_raw = []

for file_name in od_files_2019:
    tmp = pd.read_excel(file_name, header=0)
    tmp_raw.append(tmp)
    
for file_name in od_files_2020:
    tmp = pd.read_excel(file_name, header=0)
    tmp_raw.append(tmp)
    
for file_name in od_files_2021:
    tmp = pd.read_excel(file_name, header=0)
    tmp_raw.append(tmp)

In [26]:
od_raw = pd.concat(tmp_raw)
od_df = pd.DataFrame({"일자":od_raw["일자"],
                        "시도(출발)":od_raw["시도(출발)"].str.strip(),
                        "시군구(출발)":od_raw["시군구(출발)"].str.strip(),
                        "시도(도착)":od_raw["시도(도착)"].str.strip(),
                        "시군구(도착)":od_raw["시군구(도착)"].str.strip(),
                         "통행량":od_raw["통행량"]})
od_df

,일자,시도(출발),시군구(출발),시도(도착),시군구(도착),통행량
0,2019-03-13(수),서울특별시,강동구,강원도,춘천시,71
1,2019-03-13(수),서울특별시,강동구,경기도,가평군,28
2,2019-03-13(수),서울특별시,강동구,경기도,고양시,704
3,2019-03-13(수),서울특별시,강동구,경기도,과천시,218
4,2019-03-13(수),서울특별시,강동구,경기도,광명시,271
...,...,...,...,...,...,...
64,2021-03-17(수),서울특별시,강남구,인천광역시,연수구,871
65,2021-03-17(수),서울특별시,강남구,인천광역시,옹진군,2
66,2021-03-17(수),서울특별시,강남구,인천광역시,중구,745
67,2021-03-17(수),서울특별시,강남구,충청남도,아산시,22


In [27]:
od_df.reset_index(inplace=True, drop=True)

for idx, row in tqdm(od_df.iterrows()):
    od_df.iloc[idx, 0] = row["일자"][:-6]
    
od_df

0it [00:00, ?it/s]

,일자,시도(출발),시군구(출발),시도(도착),시군구(도착),통행량
0,2019-03,서울특별시,강동구,강원도,춘천시,71
1,2019-03,서울특별시,강동구,경기도,가평군,28
2,2019-03,서울특별시,강동구,경기도,고양시,704
3,2019-03,서울특별시,강동구,경기도,과천시,218
4,2019-03,서울특별시,강동구,경기도,광명시,271
...,...,...,...,...,...,...
12964,2021-03,서울특별시,강남구,인천광역시,연수구,871
12965,2021-03,서울특별시,강남구,인천광역시,옹진군,2
12966,2021-03,서울특별시,강남구,인천광역시,중구,745
12967,2021-03,서울특별시,강남구,충청남도,아산시,22


In [28]:
od_df = od_df[od_df['시도(도착)'].isin(['서울특별시', '인천광역시', '경기도'])]
od_df.reset_index(drop=True, inplace=True)
od_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12411 entries, 0 to 12410
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   일자       12411 non-null  object
 1   시도(출발)   12411 non-null  object
 2   시군구(출발)  12411 non-null  object
 3   시도(도착)   12411 non-null  object
 4   시군구(도착)  12411 non-null  object
 5   통행량      12411 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 581.9+ KB


In [29]:
od_df.to_csv('./day/통행량통합데이터_raw.csv', encoding='utf-8')

---
# 2. 피벗테이블 만들기
- 통행량은 쌍방.
- 시작, 도착 지점이 같으면 합쳐줘야한다

In [44]:
od_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12411 entries, 0 to 12410
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   일자       12411 non-null  object
 1   시도(출발)   12411 non-null  object
 2   시군구(출발)  12411 non-null  object
 3   시도(도착)   12411 non-null  object
 4   시군구(도착)  12411 non-null  object
 5   통행량      12411 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 581.9+ KB


In [51]:
order_list = ['경기도 가평군', '경기도 고양시', '경기도 과천시', '경기도 광명시', '경기도 광주시', '경기도 구리시',
       '경기도 군포시', '경기도 김포시', '경기도 남양주시', '경기도 동두천시', '경기도 부천시', '경기도 성남시 ',
       '경기도 수원시 ', '경기도 시흥시', '경기도 안산시 ', '경기도 안성시', '경기도 안양시 ', '경기도 양주시',
       '경기도 양평군', '경기도 여주시', '경기도 연천군', '경기도 오산시', '경기도 용인시 ', '경기도 의왕시',
       '경기도 의정부시', '경기도 이천시', '경기도 파주시', '경기도 평택시', '경기도 포천시', '경기도 하남시',
       '경기도 화성시', '서울특별시 강남구', '서울특별시 강동구', '서울특별시 강북구', '서울특별시 강서구',
       '서울특별시 관악구', '서울특별시 광진구', '서울특별시 구로구', '서울특별시 금천구', '서울특별시 노원구',
       '서울특별시 도봉구', '서울특별시 동대문구', '서울특별시 동작구', '서울특별시 마포구', '서울특별시 서대문구',
       '서울특별시 서초구', '서울특별시 성동구', '서울특별시 성북구', '서울특별시 송파구', '서울특별시 양천구',
       '서울특별시 영등포구', '서울특별시 용산구', '서울특별시 은평구', '서울특별시 종로구', '서울특별시 중구',
       '서울특별시 중랑구', '인천광역시 강화군', '인천광역시 계양구', '인천광역시 남동구', '인천광역시 동구',
       '인천광역시 미추홀구', '인천광역시 부평구', '인천광역시 서구', '인천광역시 연수구', '인천광역시 중구', 
              '인천광역시 옹진군']

o_li = [i.strip() for i in order_list]

In [52]:
len(o_li)

66

In [53]:
for i in range(2019,2022):
    st_df = pd.DataFrame(pd.pivot_table(data=od_df[od_df['일자'] == "{yr}-03".format(yr=i)], 
                                        index=["시도(출발)", "시군구(출발)"], 
                                        columns=["시도(도착)", "시군구(도착)"],
                                        values="통행량", aggfunc=np.sum, fill_value=0))

    s_tmp1 = [
        st_df.columns.get_level_values(0)[n] 
        +" "+ st_df.columns.get_level_values(1)[n]
        for n in range(0, len(st_df.columns.get_level_values(0)))]


    s_tmp2 = [
        st_df.index.get_level_values(0)[n] 
        +" "+ st_df.index.get_level_values(1)[n]
        for n in range(0, len(st_df.index.get_level_values(0)))]

    st_df.columns = s_tmp1
    st_df.index = s_tmp2


    ed_df = pd.DataFrame(pd.pivot_table(data=od_df[od_df['일자'] == "{yr}-03".format(yr=i)], 
                   index=["시도(도착)", "시군구(도착)"],
                   columns=["시도(출발)", "시군구(출발)"],             
                   values="통행량", aggfunc=np.sum, fill_value=0))


    e_tmp1 = [
        ed_df.columns.get_level_values(0)[n] 
        +" "+ ed_df.columns.get_level_values(1)[n]
        for n in range(0, len(ed_df.columns.get_level_values(0)))]


    e_tmp2 = [
        ed_df.index.get_level_values(0)[n] 
        +" "+ ed_df.index.get_level_values(1)[n]
        for n in range(0, len(ed_df.index.get_level_values(0)))]

    ed_df.columns = e_tmp1
    ed_df.index = e_tmp2

    st_df = st_df[o_li]
    st_df.reindex(o_li)

    ed_df = ed_df[o_li]
    ed_df.reindex(o_li)

    tmp_df = st_df + ed_df
    tmp_df.to_excel('./day/{y}년통행량.xlsx'.format(y=i), encoding='utf-8')